Import necessary packages

In [2]:
from langchain_google_vertexai import ChatVertexAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_google_vertexai import ChatVertexAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import PydanticToolsParser
import vertexai
vertexai.init(project="simcenter-llm-trials")
import datetime
from typing import Literal, Optional, Tuple

Function definitions

In [3]:
#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "VertexLang"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("\n")
    lines = [line for line in lines if not line.strip().startswith('```')]
    lines[0] = lines[0].replace(' -', '-', 1)
    return "\n".join(lines)

#invoke and run the model with the given prompt
def test_vertexai(few_shot_prompt):
    generate = ChatPromptTemplate.from_messages([("system", system), few_shot_prompt, ("human", human),])
    chat = ChatVertexAI(model_name="gemini-pro", temperature=0.5, convert_system_message_to_human=True)
    #select an output parser
    output_parser = StrOutputParser()
    chain = generate | chat | output_parser
    result = chain.invoke({}, {"tags": ["loop 001"]})
    print(remove_code_fences(result))
    print()
    return


Model Inputs

In [4]:
#create inputs to the model, telling it what needs to be done. 
#provides system message
system = context_gen("systemmsg.txt")

#provides the few shot examples
output_examples = context_gen("outputex.txt")

#provides the input examples
input_examples = context_gen("inputex.txt")

#provides the database schema
schema = context_gen("dataBaseSchema.txt")

'''change the human variable to test different inputs'''
human = ("select the blend in the xz plane between the angle 45 and 90 and then select the connected blend")


In [5]:
class SubQuery(BaseModel):
    '''Search for the geometric definition of a feature across the internet'''

    sub_query: str = Field(
        ...,
        description="The text to be used as a sub-query in the prompt.",
    )   

In [6]:
system2 = """You are an expert query generator as well as yaml generator that is able to breakdown sentences into separate words and then understand their meaning from a geometric perspective, and then use your understanding of those words for generating yaml text as shown to you in the examples.\

For example if the user says select all boltholes, then you cross-verify every word in the sentence with the database schema and then generate the yaml text as shown in the examples and if you dont understand what boltholes mean, then you will generate a subquery to look up what that word means in the geometric sense."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human),])

chat = ChatVertexAI(model_name="gemini-pro", temperature=0.5, convert_system_message_to_human=True)
chat_with_tools = chat.bind_tools([SubQuery])
parser = PydanticToolsParser(tools = [SubQuery])
query_analyzer = prompt | chat_with_tools | parser
query_analyzer.invoke({}, {"tags": ["loop 001"]})

AttributeError: 'ChatVertexAI' object has no attribute 'bind_tools'

Few shot prompts example template 

In [ ]:
examples = [
    {"input": input_examples, "output": output_examples},
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

'''Function call to run the model'''
test_vertexai(few_shot_prompt)